In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
import numpy as np
import pickle
import pandas as pd
import time
import csv

In [3]:
# pre
data_dir = "data/"
def save_obj(obj, name ):
    with open( name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(data_dir + name + '.pkl', 'rb') as f:
        return pickle.load(f)
    
# Get the training_data_df
training_data_df = load_obj("training_df_simplified_4w")
training_data_df = 
final_training_data_df = training_data_df.iloc[:,3:20]
# 0.511756 0.028259
# get the labeled data df
final_labels_df = training_data_df.iloc[:,2]

#test_df = test_df.iloc[:,2:20]

NameError: name 'final_training_data' is not defined

In [27]:
# final_training_data_df = rescale_min_max(final_training_data_df)
X=final_training_data_df
# count=0
# get the data and label
y=final_labels_df

from sklearn.model_selection import train_test_split
X_t, X_test, y_t, y_test = train_test_split(X,y)
x_train = X_t
x_test = X_test
y_train = y_t
y_test = y_test

In [ ]:
print(x_train)

In [ ]:
# x_train = np.random.random((1000, 20))
# y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10)
# x_test = np.random.random((100, 20))
# y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
import tensorflow as tf
model = Sequential()
model.add(Dense(32, input_dim=15, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))


# 选择损失函数，优化器
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.001,
    #  
    decay_steps = 1000,
    decay_rate = 1,
    staircase = False
)
def get_optimizer():
    return tf.keras.optimizers.Adam(lr_schedule)
opt = get_optimizer()
model.compile(opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          epochs=200,
          batch_size=32)
score = model.evaluate(x_test, y_test, batch_size=128)

In [43]:
test_data_df = load_obj('test_df')


In [ ]:
from tqdm import tqdm
def predict():
    """
    make the prediction using the jaccard's coefficient
    """
    result = []
    cnt=0
    for row in tqdm(test_data_df.iterrows()):
        print(row)
        row_df = pd.DataFrame(row[1]).T
        #print(row_df)
        single_result = model.predict(row_df)[0][0]
        print(model.predict(row_df))
        cnt+=1
        result.append((cnt, single_result))
    return result
result = predict()

# # save the result


'''
Description: get time
Input: 
Output: time
''' 
def nowtime():
    return time.strftime("%Y%m%d-%H%M", time.localtime())


"""
Description: Save prediction result to files
Input: (1) result
       (2) filename
Output: 
"""
def save_prediction_to_csv(result,filename):
    headers = ['id','Predicted']

    with open(filename + str(nowtime()) + ".csv", 'w', encoding = 'utf8') as f:
        f_csv = csv.writer(f)
        f_csv.writerow(headers)
        f_csv.writerows(result)
#save_prediction_to_csv(result, "shawn_ann_v1_200epoch")

2it [00:00, 13.82it/s]

(0, 0      23.0
1       3.0
2      21.0
3      29.0
4      29.0
5       0.0
6      52.0
7      32.0
8      21.0
9       0.0
10      0.0
11      0.0
12    667.0
13      0.0
14      0.0
Name: 0, dtype: float64)
[[0.10075366]]
(0, 0      74.0
1      13.0
2      71.0
3       9.0
4       9.0
5       0.0
6      83.0
7      22.0
8      71.0
9       0.0
10      0.0
11      0.0
12    666.0
13      0.0
14      0.0
Name: 0, dtype: float64)
[[0.11321968]]
(0, 0      255.000000
1       80.000000
2      205.000000
3       17.000000
4       17.000000
5        0.000000
6      272.000000
7       97.000000
8      205.000000
9        0.626606
10       0.000692
11       0.011152
12    4335.000000
13       0.462729
14       0.004624
Name: 0, dtype: float64)


6it [00:00, 15.20it/s]

[[0.11231032]]
(0, 0       511.000000
1        32.000000
2       506.000000
3        36.000000
4        36.000000
5         0.000000
6       547.000000
7        68.000000
8       506.000000
9         0.542777
10        0.000109
11        0.003670
12    18396.000000
13        0.178805
14        0.000028
Name: 0, dtype: float64)
[[0.33123592]]
(0, 0      21.0
1       5.0
2      18.0
3      46.0
4      46.0
5       0.0
6      67.0
7      51.0
8      18.0
9       0.0
10      0.0
11      0.0
12    966.0
13      0.0
14      0.0
Name: 0, dtype: float64)
[[0.10074282]]
(0, 0     51.0
1      5.0
2     50.0
3      1.0
4      1.0
5      0.0
6     52.0
7      6.0
8     50.0
9      0.0
10     0.0
11     0.0
12    51.0
13     0.0
14     0.0
Name: 0, dtype: float64)
[[0.12230849]]
(0, 0      390.000000
1      128.000000
2      327.000000
3        8.000000
4        8.000000
5        0.000000
6      398.000000
7      136.000000
8      327.000000
9        0.559616
10       0.000321
11       0.002519
12 

10it [00:00, 16.23it/s]

...

390it [00:21, 17.08it/s]

[[0.36530092]]
(0, 0      171.0
1       40.0
2      163.0
3       24.0
4       24.0
5        0.0
6      195.0
7       64.0
8      163.0
9        0.0
10       0.0
11       0.0
12    4104.0
13       0.0
14       0.0
Name: 0, dtype: float64)
[[0.11230952]]
(0, 0      101.000000
1      100.000000
2        1.000000
3       76.000000
4       76.000000
5        0.000000
6      177.000000
7      176.000000
8        1.000000
9        0.983105
10       0.001042
11       0.047337
12    7676.000000
13       1.015145
14       0.005422
Name: 0, dtype: float64)
[[0.11230937]]
(0, 0       13.000000
1        6.000000
2        9.000000
3      237.000000
4      237.000000
5        0.000000
6      250.000000
7      243.000000
8        9.000000
9        0.614289
10       0.000649
11       0.008065
12    3081.000000
13       0.422271
14       0.018228
Name: 0, dtype: float64)
[[0.10074219]]
(0, 0      445.000000
1       37.000000
2      432.000000
3       16.000000
4       16.000000
5        0.000000
6     

394it [00:22, 17.85it/s]

[[0.343248]]
(0, 0       225.000000
1        58.000000
2       193.000000
3       201.000000
4       201.000000
5         0.000000
6       426.000000
7       259.000000
8       193.000000
9         0.521385
10        0.000044
11        0.004717
12    45225.000000
13        0.190554
14        0.000056
Name: 0, dtype: float64)
[[0.11339703]]
(0, 0     11545.000000
1       195.000000
2     11502.000000
3         5.000000
4         5.000000
5         0.000000
6     11550.000000
7       200.000000
8     11502.000000
9         0.510897
10        0.000035
11        0.000173
12    57725.000000
13        0.225251
14        0.000387
Name: 0, dtype: float64)
[[0.83401644]]
(0, 0     97.0
1     35.0
2     82.0
3      1.0
4      1.0
5      0.0
6     98.0
7     36.0
8     82.0
9      0.0
10     0.0
11     0.0
12    97.0
13     0.0
14     0.0
Name: 0, dtype: float64)
[[0.09636903]]
(0, 0       1180.000000
1         71.000000
2       1175.000000
3        457.000000
4        457.000000
5          0.000

398it [00:22, 18.12it/s]